In [1]:
import os
import re
import json
import string
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 80
configuration = BertConfig()

2021-11-16 01:54:57.003026: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-16 01:54:57.009643: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-16 01:54:57.009674: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: gnode24
2021-11-16 01:54:57.009681: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: gnode24
2021-11-16 01:54:57.009751: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 440.95.1
2021-11-16 01:54:57.009781: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 440.95.1
2021-11-16 01:54:57.009788: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 440.95.1
2021-11-16 01:54:57.010186: I tensorflow/core/platform/cpu_feature_g

In [2]:
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
slow_tokenizer.save_pretrained("./bert_base_uncased/")

tokenizer = BertWordPieceTokenizer("./bert_base_uncased/vocab.txt", lowercase=True)

In [3]:
data = pd.read_csv('../train.tsv',sep='\t',header=None)
data1 = pd.read_csv('../test.tsv',sep='\t',header=None)

In [4]:
data

,0,1
0,The European Union includes how many ?,0.2
1,What are Mia Hamms accomplishment ?,0.4
2,Which form of government is still in place in ...,1.0
3,When was the canal de panama built ?,0.8
4,What color is the black box on commercial aero...,0.6
...,...,...
17495,What is the youngest college graduate ?,0.4
17496,Pros and cons of making dams ?,0.0
17497,Life and story of jose ma panganiban ?,0.0
17498,How do you apply for a super delegate position ?,1.0


In [5]:
str_list = data[0].to_list()
y_train = data[1].to_numpy()
y_train[y_train <= 0.8] = 0
y_train[y_train > 0.8] = 1

str_list1 = data1[0].to_list()
y_test = data1[1].to_numpy()
y_test[y_test <= 0.8] = 0
y_test[y_test > 0.8] = 1

In [6]:
print(len(str_list))
print(y_train.shape,y_test.shape)
print(np.unique(y_train))

17500
(17500,) (3750,)
[0. 1.]


In [7]:
def preprocess_data(sent):
  
  sent = " ".join(str(sent).split())
  tokenized_question = tokenizer.encode(sent)
  input_ids = tokenized_question.ids[:]
  token_type_ids = [1] * len(tokenized_question.ids)
  attention_mask = [1] * len(input_ids)
  padding_length = max_len - len(input_ids)
  if padding_length > 0:
    input_ids = input_ids + ([0] * padding_length)
    attention_mask = attention_mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)
  
  
  return input_ids, token_type_ids, attention_mask

In [8]:
def func():
  a_1 = []
  b_1 = []
  c_1 = []
  for i in range(len(str_list)):
    a,b,c = preprocess_data(str_list[i])
    a_1.append(a)
    b_1.append(b)
    c_1.append(c)
  
  return np.array(a_1) , np.array(b_1), np.array(c_1)


In [9]:
def func1():
  a_1 = []
  b_1 = []
  c_1 = []
  for i in range(len(str_list1)):
    a,b,c = preprocess_data(str_list1[i])
    a_1.append(a)
    b_1.append(b)
    c_1.append(c)
  
  return np.array(a_1) , np.array(b_1), np.array(c_1)

In [10]:
X_train = func()
X_test = func1()

In [11]:
encoder = TFBertModel.from_pretrained("bert-base-uncased")

input_ids = layers.Input(shape=(max_len,), name="input1", dtype=tf.int32)
token_type_ids = layers.Input(shape=(max_len,), name="input2", dtype=tf.int32)
attention_mask = layers.Input(shape=(max_len,), name="input3", dtype=tf.int32)
embedding = encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]

start_logits = layers.Flatten()(embedding)
start_logits = layers.Dense(10, name="out", use_bias=False,activation='relu')(start_logits)
final_out = layers.Dense(1,name="out1",activation='sigmoid')(start_logits)
model = keras.Model(inputs=[input_ids, token_type_ids, attention_mask],outputs=[final_out])
optimizer = keras.optimizers.Adam(lr=5e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(),metrics=["accuracy"])

In [12]:
model.layers[3].trainable = True
for i in model.layers:
  print(i.name,i.trainable)

input1 True
input3 True
input2 True
tf_bert_model True
flatten True
out True
out1 True


In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             [(None, 80)]         0                                            
__________________________________________________________________________________________________
input3 (InputLayer)             [(None, 80)]         0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             [(None, 80)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 80, 768), (N 109482240   input1[0][0]                     
______________________________________________________________________________________________

In [14]:
model.fit(X_train,y_train,epochs=1,batch_size=64)

274/274 [==============================] - 4647s 17s/step - loss: 0.4511 - accuracy: 0.7895


In [15]:
model.save_weights('./weights')

In [16]:
y_pred = model.predict(X_test)

In [17]:
y_true = y_test

In [18]:
y_pred = y_pred.reshape(y_true.shape)

print(y_pred.shape,y_true.shape)

(3750,) (3750,)


In [19]:
true_pos = np.zeros(y_true.shape)
true_neg = np.zeros(y_true.shape)
false_pos = np.zeros(y_true.shape)
false_neg = np.zeros(y_true.shape)

true_pos[(y_pred > 0.5) & (y_true == 1)] = 1
true_neg[(y_pred <= 0.5) & (y_true == 0)] = 1
false_pos[(y_pred > 0.5) & (y_true == 0)] = 1
false_neg[(y_pred <= 0.5) & (y_true == 1)] = 1

print((np.sum(true_pos) + np.sum(true_neg)) / np.sum(np.ones(y_true.shape)))

0.8226666666666667
